# linkinthe.video - Pipeline Deneyleri

Bu notebook ile video analiz pipeline'ını test ediyoruz.

**Adımlar:**
1. YouTube'dan video indir
2. Ses çıkar (ffmpeg)
3. Transkript al (Whisper)
4. Ürün çıkar (Gemini/GPT)
5. Frame'lerden ürün tanı (Vision)

---

## 0. Kurulum

In [ ]:
# Gerekli paketleri kur
!pip install -q yt-dlp openai-whisper litellm Pillow

In [ ]:
# API Key'leri ayarla (birini seç)
import os

# Google Gemini (önerilen - ucuz ve iyi)
# https://aistudio.google.com/app/apikey adresinden al
os.environ["GEMINI_API_KEY"] = "YOUR_GEMINI_API_KEY"

# OpenAI (alternatif)
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [ ]:
# Test edilecek video URL'si
# Tech review videosu seç - içinde ürün bahsi geçen
VIDEO_URL = "https://www.youtube.com/watch?v=XXXXX"  # Değiştir!

# Çalışma klasörü
WORK_DIR = "/content/linkinthe_test"
!mkdir -p {WORK_DIR}

---
## 1. Video İndir (yt-dlp)

In [ ]:
import subprocess
import os

VIDEO_PATH = f"{WORK_DIR}/video.mp4"

# Video indir (720p, boyut küçük olsun)
!yt-dlp -f "best[height<=720][ext=mp4]" -o "{VIDEO_PATH}" "{VIDEO_URL}"

# Kontrol
if os.path.exists(VIDEO_PATH):
    size_mb = os.path.getsize(VIDEO_PATH) / (1024 * 1024)
    print(f"✅ Video indirildi: {size_mb:.1f} MB")
else:
    print("❌ Video indirilemedi!")

---
## 2. Ses Çıkar (ffmpeg)

In [ ]:
AUDIO_PATH = f"{WORK_DIR}/audio.wav"

# ffmpeg ile ses çıkar
!ffmpeg -i "{VIDEO_PATH}" -vn -acodec pcm_s16le -ar 16000 -ac 1 "{AUDIO_PATH}" -y -loglevel error

# Kontrol
if os.path.exists(AUDIO_PATH):
    size_mb = os.path.getsize(AUDIO_PATH) / (1024 * 1024)
    print(f"✅ Ses çıkarıldı: {size_mb:.1f} MB")
else:
    print("❌ Ses çıkarılamadı!")

---
## 3. Transkript Al (Whisper)

In [ ]:
import whisper
import time

# Model yükle (base = hızlı, medium = dengeli, large = en iyi)
print("Model yükleniyor...")
model = whisper.load_model("base")  # Colab için 'base' yeterli

# Transkript al
print("Transkript alınıyor...")
start = time.time()
result = model.transcribe(AUDIO_PATH)
elapsed = time.time() - start

print(f"✅ Transkript tamamlandı ({elapsed:.1f} saniye)")
print(f"Dil: {result['language']}")
print(f"Uzunluk: {len(result['text'])} karakter")
print("\n" + "="*50)
print(result['text'][:2000])  # İlk 2000 karakter
print("="*50)

In [ ]:
# Timestamp'li segmentlere bak
print("İlk 10 segment (timestamp'li):")
print("-" * 50)
for seg in result['segments'][:10]:
    start = seg['start']
    end = seg['end']
    text = seg['text'].strip()
    print(f"[{start:05.1f} - {end:05.1f}] {text}")

In [ ]:
# Transkripti kaydet
TRANSCRIPT_PATH = f"{WORK_DIR}/transcript.txt"
with open(TRANSCRIPT_PATH, "w") as f:
    f.write(result['text'])
print(f"✅ Transkript kaydedildi: {TRANSCRIPT_PATH}")

---
## 4. Ürün Çıkar (LLM)

In [ ]:
import litellm
import json

PRODUCT_EXTRACTION_PROMPT = """
Bu bir YouTube tech review videosunun transkripti.
Videoda bahsedilen TÜM ürünleri bul.

Her ürün için:
- name: Ürün adı (marka + model)
- category: Kategori (örn: tripod, mikrofon, kamera, laptop, aksesuar)
- confidence: Ne kadar eminsin (0-1 arası)
- context: Üründen nasıl bahsedildi (1 cümle)

JSON formatında dön. Örnek:
{
  "products": [
    {"name": "Sony A7IV", "category": "kamera", "confidence": 0.95, "context": "Ana kamera olarak kullanıyor"},
    {"name": "Rode VideoMic", "category": "mikrofon", "confidence": 0.8, "context": "Ses kaydı için önerdi"}
  ]
}

Transkript:
---
{transcript}
---

Sadece JSON dön, başka bir şey yazma.
"""

# Gemini ile ürün çıkar
response = litellm.completion(
    model="gemini/gemini-1.5-flash",
    messages=[{
        "role": "user",
        "content": PRODUCT_EXTRACTION_PROMPT.format(transcript=result['text'][:8000])
    }],
    temperature=0.1,  # Deterministik sonuç için düşük
)

llm_output = response.choices[0].message.content
print("LLM Çıktısı:")
print(llm_output)

In [ ]:
# JSON parse et
try:
    # Markdown code block temizle
    clean_output = llm_output
    if "```json" in clean_output:
        clean_output = clean_output.split("```json")[1].split("```")[0]
    elif "```" in clean_output:
        clean_output = clean_output.split("```")[1].split("```")[0]
    
    products_data = json.loads(clean_output.strip())
    products = products_data.get("products", products_data)
    
    print(f"\n✅ {len(products)} ürün bulundu:\n")
    for i, p in enumerate(products, 1):
        conf = p.get('confidence', 0)
        emoji = "🟢" if conf > 0.8 else "🟡" if conf > 0.5 else "🔴"
        print(f"{i}. {emoji} {p.get('name', 'N/A')}")
        print(f"   Kategori: {p.get('category', 'N/A')}")
        print(f"   Güven: {conf}")
        print(f"   Bağlam: {p.get('context', 'N/A')}")
        print()
except json.JSONDecodeError as e:
    print(f"❌ JSON parse hatası: {e}")
    print(f"Ham çıktı: {llm_output}")

---
## 5. Frame Çıkar (Video'dan)

In [ ]:
import subprocess
import glob

FRAMES_DIR = f"{WORK_DIR}/frames"
!mkdir -p {FRAMES_DIR}

# Her 30 saniyede bir frame çıkar
!ffmpeg -i "{VIDEO_PATH}" -vf "fps=1/30" "{FRAMES_DIR}/frame_%04d.jpg" -loglevel error

frames = sorted(glob.glob(f"{FRAMES_DIR}/*.jpg"))
print(f"✅ {len(frames)} frame çıkarıldı")

# İlk birkaç frame'i göster
from IPython.display import Image, display
for frame in frames[:3]:
    print(f"\n{frame}")
    display(Image(filename=frame, width=400))

---
## 6. Frame'den Ürün Tanı (Vision)

In [ ]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def analyze_frame(image_path, known_products=None):
    """Frame'deki ürünleri tanı."""
    image_data = encode_image(image_path)
    
    context = ""
    if known_products:
        context = f"\nVideoda bahsedilen ürünler: {', '.join(known_products)}"
    
    prompt = f"""Bu bir tech review videosundan alınmış frame.
Görselde görünen ürünleri tanımla.{context}

JSON formatında dön:
{{"products": [{{"name": "Ürün adı", "confidence": 0.9, "location": "frame'de nerede"}}]}}

Sadece JSON dön."""
    
    response = litellm.completion(
        model="gemini/gemini-1.5-flash",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}
                }
            ]
        }],
        temperature=0.1,
    )
    
    return response.choices[0].message.content

# İlk frame'i analiz et
if frames:
    print(f"Analiz edilen frame: {frames[0]}")
    display(Image(filename=frames[0], width=400))
    
    # Bilinen ürün isimlerini ver (transkriptten)
    known = [p.get('name') for p in products] if 'products' in dir() else None
    
    vision_result = analyze_frame(frames[0], known)
    print("\nVision Analizi:")
    print(vision_result)

In [ ]:
# Tüm frame'leri analiz et (maliyet: ~$0.01/frame Gemini ile)
all_vision_products = []

print(f"{len(frames)} frame analiz ediliyor...")
for i, frame in enumerate(frames):
    print(f"Frame {i+1}/{len(frames)}...", end=" ")
    try:
        result = analyze_frame(frame)
        # Parse et
        clean = result
        if "```" in clean:
            clean = clean.split("```")[1].replace("json", "").strip()
        data = json.loads(clean)
        prods = data.get("products", [])
        all_vision_products.extend(prods)
        print(f"✅ {len(prods)} ürün")
    except Exception as e:
        print(f"❌ Hata: {e}")

print(f"\nToplam: {len(all_vision_products)} ürün tespiti")

---
## 7. Sonuçları Birleştir

In [ ]:
from collections import defaultdict

# Transkript + Vision sonuçlarını birleştir
combined = defaultdict(lambda: {"sources": [], "confidence": 0})

# Transkriptten gelenler
if 'products' in dir():
    for p in products:
        name = p.get('name', '').lower()
        combined[name]["name"] = p.get('name')
        combined[name]["sources"].append("audio")
        combined[name]["confidence"] = max(
            combined[name]["confidence"], 
            p.get('confidence', 0)
        )
        combined[name]["category"] = p.get('category', '')

# Vision'dan gelenler
for p in all_vision_products:
    name = p.get('name', '').lower()
    combined[name]["name"] = p.get('name')
    if "video" not in combined[name]["sources"]:
        combined[name]["sources"].append("video")
    combined[name]["confidence"] = max(
        combined[name]["confidence"],
        p.get('confidence', 0)
    )

# Sonuçları göster
print("="*60)
print("BİRLEŞTİRİLMİŞ SONUÇLAR")
print("="*60)

sorted_products = sorted(
    combined.values(), 
    key=lambda x: x['confidence'], 
    reverse=True
)

for i, p in enumerate(sorted_products, 1):
    sources = ", ".join(p['sources'])
    conf = p['confidence']
    emoji = "🟢" if conf > 0.8 else "🟡" if conf > 0.5 else "🔴"
    print(f"{i}. {emoji} {p.get('name', 'N/A')}")
    print(f"   Kaynaklar: [{sources}]")
    print(f"   Güven: {conf}")
    print()

---
## 8. Maliyet Hesabı

In [ ]:
# Tahmini maliyet hesapla
transcript_tokens = len(result['text']) / 4  # ~4 karakter = 1 token
vision_calls = len(frames)

# Gemini Flash fiyatları (Kasım 2024)
# Input: $0.075/1M token, Output: $0.30/1M token
# Vision: ~$0.0025/image

llm_cost = (transcript_tokens / 1_000_000) * 0.075 + (500 / 1_000_000) * 0.30
vision_cost = vision_calls * 0.0025
total_cost = llm_cost + vision_cost

print("💰 TAHMİNİ MALİYET")
print("="*40)
print(f"Transkript uzunluğu: {len(result['text']):,} karakter")
print(f"Frame sayısı: {vision_calls}")
print(f"")
print(f"LLM (ürün çıkarma): ${llm_cost:.4f}")
print(f"Vision ({vision_calls} frame): ${vision_cost:.4f}")
print(f"-" * 40)
print(f"TOPLAM: ${total_cost:.4f}")
print(f"")
print(f"📊 1000 video için: ${total_cost * 1000:.2f}")

---
## Sonuç ve Notlar

### Çalıştı mı?
- [ ] Video indirme OK
- [ ] Ses çıkarma OK  
- [ ] Whisper transkript OK
- [ ] LLM ürün çıkarma OK
- [ ] Vision ürün tanıma OK

### Kalite nasıl?
- Whisper doğruluğu: /10
- LLM ürün bulma: /10
- Vision tanıma: /10

### Kaçan ürünler:
- ...

### Yanlış bulunanlar:
- ...

### Notlar:
- ...